Sqoop query
===

* Última modificación: Mayo 19, 2022

Descarga de datos
---

In [1]:
file_name = "drivers.csv"
url = "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/"
!wget --quiet {url + file_name} -P /tmp/

Preparación de la base de datos
---

In [2]:
import sqlite3

conn = sqlite3.connect("sample.db")
cur = conn.cursor()

conn.executescript(
    """
    DROP TABLE IF EXISTS drivers;

    CREATE TABLE drivers (
        driverId       INT,
        name           STRING,
        ssn            STRING,
        location       STRING,
        certified      STRING,
        wage_plan      STRING
    );
    """
)
conn.commit()

with open("/tmp/drivers.csv", "rt") as f:
    data = f.readlines()

data = [line.replace('\n', '') for line in data]
data = [line.split(",") for line in data]
data = [tuple(line) for line in data]
data = data[1:]

cur.executemany("INSERT INTO drivers VALUES (?,?,?,?,?,?)", data)
conn.commit()
conn.close()

In [3]:
!sqlite3 sample.db "select * from drivers" | head

10|George Vetticaden|621011971|244-4532 Nulla Rd.|N|miles
11|Jamie Engesser|262112338|366-4125 Ac Street|N|miles
12|Paul Coddin|198041975|Ap #622-957 Risus. Street|Y|hours
13|Joe Niemiec|139907145|2071 Hendrerit. Ave|Y|hours
14|Adis Cesir|820812209|Ap #810-1228 In St.|Y|hours
15|Rohit Bakshi|239005227|648-5681 Dui- Rd.|Y|hours
16|Tom McCuch|363303105|P.O. Box 313- 962 Parturient Rd.|Y|hours
17|Eric Mizell|123808238|P.O. Box 579- 2191 Gravida. Street|Y|hours
18|Grant Liu|171010151|Ap #928-3159 Vestibulum Av.|Y|hours
19|Ajay Singh|160005158|592-9430 Nonummy Avenue|Y|hours


Query
--

In [4]:
%%writefile query.sh
sqoop eval \
    --connect jdbc:sqlite:sample.db \
    --driver org.sqlite.JDBC \
    --query "SELECT * FROM drivers LIMIT 3"

Writing query.sh


In [5]:
!bash query.sh

Please set $HBASE_HOME to the root of your HBase installation.
Please set $HCAT_HOME to the root of your HCatalog installation.
Please set $ACCUMULO_HOME to the root of your Accumulo installation.
Please set $ZOOKEEPER_HOME to the root of your Zookeeper installation.
-------------------------------------------------------------------------------------------------------------------------------------------
| driverId             | name                 | ssn                  | location             | certified            | wage_plan            | 
-------------------------------------------------------------------------------------------------------------------------------------------
| 10                   | George Vetticaden    | 621011971            | 244-4532 Nulla Rd.   | N                    | miles                | 
| 11                   | Jamie Engesser       | 262112338            | 366-4125 Ac Street   | N                    | miles                | 
| 12                   | Paul

In [6]:
!rm *.db *.sh *.java